# 1. Import Library

# 資料準備

In [1]:

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model,Model
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.layers.wrappers import Bidirectional
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,GRU,Conv1D, MaxPooling1D,GlobalMaxPooling1D,SpatialDropout1D
from keras.utils import np_utils
from datetime import datetime,timedelta

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.optimizers import adam
import tensorflow.keras as keras
import numpy as np
import requests
import pandas as pd

# import urllib.request
import os
import tarfile
np.random.seed(10)


/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#os.chdir("../")
#os.getcwd()

In [3]:
#from data_utils import cut_to_word,covert_sequences,get_word2vec_embeddings
#from data_utils import get_average_word2vec,produce_cbowfile,JiebaSegmentor,data_reduction,seperatedata
from data_utils import *
from data_utils import JiebaSegmentor

In [4]:
jieba_dict_path1 = "jieba/dict_taiwan.txt"
jieba_dict_path2 = "jieba/userdict.txt"
jieba_dict_path3 = "jieba/dict.txt.big"
jieba_dict_path4 = "jieba/dict.txt.small"
jieba_dict_path5 = "jieba/dict_system.txt"
jieba_stopwords_path = "jieba/stopwords.txt"

js = JiebaSegmentor(dict_path=jieba_dict_path1,
                    userdict=[jieba_dict_path2,jieba_dict_path5],
                    stopwords=True,
                    stopwords_path=jieba_stopwords_path)

print(js)
# js = JiebaSegmentor(jieba_dict_path1, [jieba_dict_path2, jieba_dict_path3, jieba_dict_path4])

Building prefix dict from /project/at081-group3/Project_19999/19999_2RNN/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.u5f4d4d2a4ebf6cacbf248ccc4ee696aa.cache
Loading model cost 0.516 seconds.


jieba/userdict.txt


Prefix dict has been built succesfully.


jieba/dict_system.txt


In [5]:
test_cut_raw_0 = js.lcut('BPM系統有問題', cut_type='df')
# test_cut_raw_0
for x in test_cut_raw_0:
    print (x)

word


In [6]:
test_cut_raw_0 = js.pseg_lcut('BPM系統有問題，無法 使用')
test_cut_raw_0

,word,tag
0,BPM系統,ns
1,無法,n
2,,x
3,使用,n


In [7]:
test_cut_raw_0 = js.lcut('R&D系統有問題，無法使用')
test_cut_raw_0

,word
0,R&D
1,系統
2,無法使用


In [8]:
#先讀取所有文章建立字典，限制字典的數量為nb_words=2000

In [11]:
df = pd.read_csv('data/19999_question_category_a_v11_test.csv')#usecols=['description','category_a', 'category_a_target']

In [12]:
df.head(10)

,description,category_a,category_b,category_c,category_d,category,description_clean,category_target
0,"無法會員登入,顯示訊息:無效的帳號或使用者不存在。帳號:96122401",CAMP,CAMP,CAMP,CAMP,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,因今天有事請假，要補登先前的加班，但開啟camp後，出勤表單顯示的內容看起來像測試的，沒有最...,CAMP,CAMP,CAMP,CAMP,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4
2,"手機簽核BPM有問題,都顯示亂碼 #18227",CAMP,CAMP,CAMP,CAMP,資訊系統,簽核BPM有問題 都顯示亂碼,4
3,無法在手機上面使用CAMP進行表單簽核動作。分機：17132,CAMP,CAMP,CAMP,CAMP,資訊系統,無法在上面使用CAMP進行表單簽核動作,4
4,"出差同仁無法登入camp, 委請同事詢問",CAMP,CAMP,CAMP,CAMP,資訊系統,出差同仁無法登入camp 委請同事詢問,4
5,"請問一下，為什麼我進入CAMP，在會員中心點選""應用程式""卻找不到""研發服務雲""可點選? ...",CAMP,CAMP,CAMP,CAMP,資訊系統,請問一下 我進入CAMP 在會員中心點選 應用程式 卻找不到 研發服務雲 可點選 該解決,4
6,用手機CAMP要簽核電腦外帶攜出申請單，出現應用程式發生錯誤 #15189,CAMP,CAMP,CAMP,CAMP,資訊系統,用CAMP要簽核電腦外帶攜出申請單 出現應用程式發生錯誤,4
7,研發服務雲 > 偕同開發 > 新增專案 : 無法新增 J51 or J51D 專案 for ...,CAMP,CAMP,CAMP,CAMP,資訊系統,研發服務雲 偕同開發 新增專案 無法新增 J51 or J51D 專案 for ...,4
8,"進入 CAMP 「研發服務雲」的icon 沒有進一步的資訊 , 畫面空白,沒有發包選單",CAMP,CAMP,CAMP,CAMP,資訊系統,進入 CAMP 「研發服務雲」的icon 沒有進一步的資訊 畫面空白 沒有發包選單,4
9,"在CAMP平台點選""應用程式""找不到""研發服務雲""可以點選，請問該如何解決?",CAMP,CAMP,CAMP,CAMP,資訊系統,在CAMP平臺點選 應用程式 找不到 研發服務雲 可以點選,4


In [13]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8384 entries, 0 to 8383
Data columns (total 8 columns):
description          8384 non-null object
category_a           8384 non-null object
category_b           8384 non-null object
category_c           8384 non-null object
category_d           8384 non-null object
category             8384 non-null object
description_clean    8384 non-null object
category_target      8384 non-null int64
dtypes: int64(1), object(7)
memory usage: 524.1+ KB


In [14]:
#df[df['description'].str.len()<5].category_a_target.value_counts()/df.category_a_target.value_counts()



In [15]:
# index to categories mapping
mapping = df.category.astype('category').cat.categories
mapping

Index(['Outlook與郵件', 'SAP', '其他', '網路', '資訊系統'], dtype='object')

In [16]:
#df=data_reduction(df)
#df = df.drop( labels = df[(df['category_target'] != 6) & (df['category_target'] != 0) & (df['category_target'] != 3) 
#                          & (df['category_target'] != 1) & (df['category_target'] != 7)].index, axis = 0 )

In [17]:
df.groupby(["category"]).size()

category
Outlook與郵件    1778
SAP           1443
其他            1470
網路            2314
資訊系統          1379
dtype: int64

In [18]:
print (cut_to_word('可以幫我聯絡'.strip(),js))
#df['cut_words'] = df['description_add_category'].apply(lambda s: cut_to_word(s.strip(),js))
#df['cut_words'] = df['description_add_category'].apply(lambda s: cut_to_word(s.strip(),js))
#df['cut_words_clean'] = df['description_clean'].apply(lambda s: cut_to_word(s.strip(),js))
#df['cut_words_clean'] = df['description_clean'].apply(lambda s: cut_to_word(s.strip(),js))

#df['cut_words'] = df['description'].apply(lambda s: cut_to_word(s.strip(),js))
#df['cut_words'] = df['description'].apply(lambda s: cut_to_word(s.strip(),js))
df['cut_words'] = df['description_clean'].apply(lambda s: cut_to_word(s.strip(),js))
#df['cut_words'] = df['description_clean_add_category_abcd'].apply(lambda s: cut_to_word(s.strip(),js))


聯絡


TODO: 拿掉時間,人名

In [19]:
data,word2vec=produce_cbowfile(df.description_clean ,jieba_stopwords_path)

In [20]:
#data
df[:5]

,description,category_a,category_b,category_c,category_d,category,description_clean,category_target,cut_words
0,"無法會員登入,顯示訊息:無效的帳號或使用者不存在。帳號:96122401",CAMP,CAMP,CAMP,CAMP,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4,"無法,會員,無法會員,登入,顯示,訊息,無效,帳號,不存,帳號"
1,因今天有事請假，要補登先前的加班，但開啟camp後，出勤表單顯示的內容看起來像測試的，沒有最...,CAMP,CAMP,CAMP,CAMP,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4,"有事,請假,補登,先前,加班,開啟,camp,出勤,表單,顯示,內容,測試,時間,使用,登出..."
2,"手機簽核BPM有問題,都顯示亂碼 #18227",CAMP,CAMP,CAMP,CAMP,資訊系統,簽核BPM有問題 都顯示亂碼,4,"簽核,BPM,顯示,亂碼"
3,無法在手機上面使用CAMP進行表單簽核動作。分機：17132,CAMP,CAMP,CAMP,CAMP,資訊系統,無法在上面使用CAMP進行表單簽核動作,4,"無法,上面,無法上面,使用,CAMP,進行,表單,簽核,動作"
4,"出差同仁無法登入camp, 委請同事詢問",CAMP,CAMP,CAMP,CAMP,資訊系統,出差同仁無法登入camp 委請同事詢問,4,"出差,同仁,無法登入,camp,委請,同事,詢問"


In [21]:
#word2vec.wv.most_similar(positive=['outlook'], negative=['網路'])

In [22]:
embeddingMatrix=get_embedding_martrix()

In [23]:
#print(data.items())

In [24]:
x = df
y = df.category_target
#将数据划分为训练集和验证集
#注意要打乱数据，因为原始数据是分类排好序的
x_train,x_test,y_train,y_test =seperatedata(x,y)

In [25]:
y_train.value_counts()

3    1759
0    1358
2    1078
1    1066
4    1027
Name: category_target, dtype: int64

In [26]:
y_test.value_counts()

3    555
0    420
2    392
1    377
4    352
Name: category_target, dtype: int64

In [27]:
#le = LabelEncoder()  
#y_test = le.fit_transform(y_test) # 這邊Y.shape = (84020, )
#y_test = y_test.reshape(-1,1) # 將Y的shape轉換成： Y.shape= (84020, 1)

In [28]:
# label 做 onehot
y_one_hot,num_classes = labelEncoding(y_train)

[0. 1. 0. 0. 0.]
num_classes = 5


In [29]:
# label 做 onehot
#training_embeddings = get_word2vec_embeddings(word2vec, x_train, generate_missing=True)

In [30]:
y_test.head(10)

2865    3
8175    2
5933    0
4857    3
6859    0
2031    4
1056    3
2858    3
7216    2
4038    3
Name: category_target, dtype: int64

In [31]:
df_test_y=y_test
df_test_x=x_test
df_test_x.head(10)

,description,category_a,category_b,category_c,category_d,category,description_clean,category_target,cut_words
2865,使用者反應無法連線 Internet。(Ext.15518),網管,網路,無法連到大陸或其他分公司,連線分公司網路問題,網路,無法連線 Internet,3,"無法連線,Internet"
8175,分機: 15354 問題: 印表機的文件都在等候列印中，無法列印,網管,作業系統與印表機,印表機問題,印表機無法列印,其他,印表機的文件都在等候列印中 無法列印,2,"印表機,文件,等候,列印,無法,列印,無法列印"
5933,Microsot Outlook 認證視窗不斷跳出，要求填列相關認證資訊(QUANTA\Qu...,網管,Outlook與郵件,Outlook,其他問題,Outlook與郵件,Microsot Outlook 認證視窗不斷跳出 要求填列相關認證資訊 QUANTA\Qu...,0,"Microsot,Outlook,認證,視窗,不斷,跳出,要求,填列,認證,資訊,QUANT..."
4857,"有線網路無法連線 , Proxy憑證無法辨識",網管,網路,電腦網路不通,網路不斷出現驗證訊息,網路,有線網路無法連線 Proxy憑證無法辨識,3,"網路無法連線,Proxy,憑證,無法,辨識,無法辨識"
6859,郵件異常，開了網頁後郵件就無法收發，甚至無法收信 #3825-623106,網管,Outlook與郵件,Outlook,郵件無法收發,Outlook與郵件,郵件異常 開了網頁後郵件就無法收發 無法收信,0,"郵件,異常,網頁,郵件,無法收發,無法收信"
2031,請問一下當初QSIT設計時，上傳檔案的大小限制是多少呢 還有command欄位好像有字串長...,R&D資訊系統,測試相關服務,【QSIT】系統整合測試系統,相關操作詢問,資訊系統,請問一下當初QSIT設計時 上傳檔案的大小限制是呢 command欄位有字串長度限制 是可...,4,"請問,當初,QSIT,計時,上傳,檔案,限制,command,欄位,字串,長度,限制,填多長,字串"
1056,使用Open VPN 時連上MyBPM 首頁但無法再Link進去 請協助處理 EXT:15306,網管,VPN連線,SSLVPN問題,無法登入,網路,使用Open VPN 時連上MyBPM 首頁但無法再Link,3,"使用,Open,VPN,連上,My,BPM,首頁,無法,Link"
2858,無法連線到上海模具廠的FTP。,網管,網路,無法連到大陸或其他分公司,連線分公司網路問題,網路,無法連線到模具廠的FTP,3,"無法連線,模具廠,FTP"
7216,Outlook可以登入及MVPN也可以使用，但是商用Skype無法登入使用。,網管,商務用Skype,無法登入,無法登入,其他,Outlook可以登入及也可以使用 商用Skype無法登入使用,2,"Outlook,登入,使用,商用,Skype,無法登入,使用"
4038,因工作需要，下在中國人民共和國知識產權局的專利文件，因故無法下載，錯誤訊息如下： 錯誤碼：...,網管,網路,無法連到某台Server,無法連到某些Server,網路,因工作需要 下在中國人民共和國知識產權局的專利文件 因故無法下載 錯誤訊息 錯誤碼 64...,3,"工作,中國人,共和國,知識產權,專利,文件,因故,無法,下載,無法下載,錯誤訊息,錯誤碼,6..."


In [32]:
# 標點符號過濾
WORD_FILTERS = '!"#$&()*+,-./:;<=>?@[\\]^_{|}~\t\n'
# 字典數量
#只考虑数据集中前10000个最常见的单词
NUM_WORDS = 5000
# 向量長度
#100个单词后截断评论，这个参数非常影响准确度
MAX_LEN = 30

embedding_dim=256

x_train2=x_train
#将整数列表转化成形状为（samples，maxlen）的二维整数张量
x_train,train_word_index = covert_sequences(x_train.cut_words,NUM_WORDS, MAX_LEN)
x_test,train_word_index = covert_sequences(x_test.cut_words,NUM_WORDS, MAX_LEN)
x_train2,train_word_index = covert_sequences(x_train2.cut_words,NUM_WORDS, MAX_LEN)

Found 8741 unique tokens.
Found 4253 unique tokens.
Found 8741 unique tokens.


In [33]:
#https://www.kaggle.com/jannesklaas/17-nlp-and-word-embeddings
print('Token for "the"',train_word_index.get("系統"))

Token for "the" 7


In [34]:
x_test

array([[   0,    0,    0, ...,    0,   24,  493],
       [   0,    0,    0, ...,    1,   68,  147],
       [   0,    0,    0, ..., 1107,  389, 1108],
       ...,
       [   0,    0,    0, ...,  353,    3,  378],
       [   0,    0,    0, ...,  166,   80,    3],
       [   0,    0,    0, ...,    9,   10,   51]], dtype=int32)

In [35]:
#train_word_index

In [36]:
train_embedding_weights = np.zeros((len(train_word_index)+1, embedding_dim))

for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(embedding_dim)
    #if word in word2vec:
    #    print(word2vec[word])
print(train_embedding_weights)
print("-----------=====-----------")
print(train_embedding_weights.shape)
# 字典數量
#NUM_WORDS = train_embedding_weights.shape[0]

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.43198535 -0.45890471  0.1615601  ...  0.06966524  0.11873438
  -0.45133391]
 [ 0.3179037  -0.33691368  0.11977738 ...  0.05055702  0.08750065
  -0.33338067]
 ...
 [ 0.13899864  0.46566343  0.96458152 ...  0.17579288  0.17860688
   0.64794416]
 [ 0.34751926  0.27848674  0.872602   ...  0.47948263  0.72211493
   0.36880417]
 [ 0.44574838  0.74316155  0.89275476 ...  0.62990687  0.77434593
   0.75087547]]
-----------=====-----------
(8742, 256)


In [37]:
nb_words = min(NUM_WORDS, len(train_word_index)) # How many words are there actually

embedding_matrix = np.zeros((nb_words, embedding_dim))

# The vectors need to be in the same position as their index. 
# Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on

# Loop over all words in the word index
for word, i in train_word_index.items():
    # If we are above the amount of words we want to use we do nothing
    if i >= NUM_WORDS: 
        continue
    
    if word in word2vec:
        embedding_vector=word2vec[word]
        embedding_matrix[i] = embedding_vector
    # Get the embedding vector for the word
    #embedding_vector = df.cut_words.get(word)
    #print(word)
    # If there is an embedding vector, put it in the embedding matrix
    #if embedding_vector is not None: 
    #    embedding_matrix[i] = embedding_vector

In [38]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.43198535, -0.45890471,  0.1615601 , ...,  0.06966524,
         0.11873438, -0.45133391],
       [ 0.3179037 , -0.33691368,  0.11977738, ...,  0.05055702,
         0.08750065, -0.33338067],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [39]:
print (x_train)
# x_train

[[   0    0    0 ...  285  562   37]
 [   0    0    0 ...  849   24  411]
 [   0    0    0 ...   75 1401   22]
 ...
 [   0    0    0 ...   12    2   32]
 [   0    0    0 ...  932  154  688]
 [   0    0    0 ...  143  322  763]]


In [40]:
print (x_test.shape)
x_test

(2096, 30)


array([[   0,    0,    0, ...,    0,   24,  493],
       [   0,    0,    0, ...,    1,   68,  147],
       [   0,    0,    0, ..., 1107,  389, 1108],
       ...,
       [   0,    0,    0, ...,  353,    3,  378],
       [   0,    0,    0, ...,  166,   80,    3],
       [   0,    0,    0, ...,    9,   10,   51]], dtype=int32)

In [41]:
x_train.shape, y.shape, x_test.shape

((6288, 30), (8384,), (2096, 30))

In [42]:
y_one_hot[:5]

array([[0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

# 建立模型

In [43]:
print(NUM_WORDS)
print(num_classes)
print(MAX_LEN)
print(embeddingMatrix.shape)

5000
5
30
(1815, 256)


In [44]:
def BidLSTM(num_classes, vocab_size, sentence_max_len,embedding_dim):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim, 
                        input_length=sentence_max_len))
    model.add(Bidirectional(LSTM(128,implementation=2)))
    model.add(Dropout(0.5))
    #model.add(Flatten())
    model.add(Dense(256,activation='tanh'))
    model.add(Dropout(0.5))
    #model.add(Dense(50,activation='tanh'))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [45]:
def LSTM(num_classes, vocab_size, sentence_max_len,embedding_dim):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=sentence_max_len))
    model.add(SpatialDropout1D(0.7))
    model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
    model.add(Dense(50,activation='tanh'))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [46]:
from keras.layers import Layer
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
 
 
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """
 
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
 
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
 
        super(AttentionWithContext, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None
 
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
 
        if self.bias:
            uit += self.b
 
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
 
        a = K.exp(ait)
 
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
 
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
 
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [47]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras import backend as K
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [48]:
#https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-672-lb
#NUM_WORDS=embedding_matrix.shape[0]
from keras.layers import core,BatchNormalization
#CuDNNLSTM比較厲害
from keras.layers import CuDNNLSTM
def BidCuDNNLSTM(num_classes, vocab_size, sentence_max_len,embedding_dim):
    SEQ_LEN = 40  # magic number - length to truncate sequences of words
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim, 
                        #weights=[embedding_matrix],#需要研究如何用, 結果會變差
                        input_length=sentence_max_len,trainable = False))
    model.add(Bidirectional(CuDNNLSTM(256, return_sequences=True)))
    model.add(Bidirectional(CuDNNLSTM(128,return_sequences=True)))
    model.add(Attention(SEQ_LEN))
    #model.add(AttentionWithContext())
    model.add(Dropout(0.5))
    #model.add(Flatten())
    #model.add(Dense(256,activation='tanh'))
    #model.add(Dropout(0.5))
    #model.add(Dense(50,activation='tanh'))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [49]:
#https://www.kaggle.com/takuok/bidirectional-lstm-and-attention-lb-0-043
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#速度有點慢，不建議
def BidLstm(num_classes, vocab_size, sentence_max_len,embedding_dim):
    inp = Input(shape=(sentence_max_len, ))
    x = Embedding(vocab_size, embedding_dim, #weights=[embedding_matrix],
                  trainable=False)(inp)
    x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(x)
    x = Attention(sentence_max_len)(x)
    x = Dropout(0.5)(x)
    #x = Dense(256, activation="tanh")(x)
    #x = Dropout(0.5)(x)
    #x = Dense(num_classes, activation="sigmoid")(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=inp, outputs=x)
    print(model.summary())
    return model

In [50]:
from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
#https://www.kaggle.com/fareise/multi-head-self-attention-for-text-classification

def CNNText(num_classes, vocab_size, max_sequence_length, embedding_dim):
    
    model = Sequential()
    inp = Input(shape=(max_sequence_length,))
    x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix])(inp)
    x = Reshape((max_sequence_length, embedding_dim, 1))(x)
    
    x = Attention(vocab_size)(x)  #output: [batch_size, time_step, nb_head*size_per_head]
    x = GlobalAveragePooling1D()(x)
    
    x = Dense(num_classes, activation='sigmsoftmaxoid')(x)
    
    model = Model(inputs = inp, outputs = x)
    print(model.summary())
    return model

In [51]:
#https://mlwhiz.com/blog/2019/03/09/deeplearning_architectures_text_classification/
from keras.layers import Reshape
from keras.layers.pooling import MaxPool2D
def CCN_Conv2D(num_classes, vocab_size, max_sequence_length, embedding_dim):
    filter_sizes = [1,2,3,5]
    num_filters = vocab_size

    inp = Input(shape=(max_sequence_length,))
    x = Embedding(vocab_size, embedding_dim#, weights=[embedding_matrix]
                 )(inp)
    x = Reshape((max_sequence_length, embedding_dim, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_dim),
                                     kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(max_sequence_length - filter_sizes[i] + 1, 1))(conv))

    z = concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    #outp = Dense(1, activation="sigmoid")(z)
    outp = Dense(num_classes, activation='sigmsoftmaxoid')(z)
    model = Model(inputs=inp, outputs=outp)
    print(model.summary())
    return model

In [52]:
from keras.layers.merge import concatenate, add
from keras.layers import CuDNNGRU
#https://www.kaggle.com/kakiac/deep-learning-4-text-classification-cnn-bi-lstm

In [53]:
#https://www.kaggle.com/ngyptr/multi-class-classification-with-lstm
#https://www.kaggle.com/eashish/bidirectional-gru-with-convolution
#https://stackoverflow.com/questions/53904688/using-keras-to-build-a-lstmconv2d-model
#效果也不錯
from keras.layers.pooling import GlobalAveragePooling1D
def GRU(num_classes, vocab_size, max_sequence_length, embedding_dim):
    inp = Input(shape = (max_sequence_length,))
    x = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], trainable = False)(inp)
    
    #組合1
    #x = SpatialDropout1D(0.7)(x)
    #x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)  
    
    #組合2
    #https://www.kaggle.com/eashish/bidirectional-gru-with-convolution
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(128,return_sequences=True))(x)
    x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    
    #組合3
    #https://www.kaggle.com/taindow/simple-cudnngru-python-keras#Model-Architecture
    #x = SpatialDropout1D(0.2)(x)
    #x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)   
    #x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    
    att = Attention(38)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x) 
    conc = concatenate([att,avg_pool, max_pool])
    
    output = Dropout(0.5)(conc)
    output = Dense(units=64)(output)
    output = Activation('relu')(output)
    preds=Dense(num_classes, activation='softmax')(output)
    model = Model(inputs = inp, outputs = preds)
    print(model.summary())
    return model;

In [54]:
def model_lstm_du(num_classes, vocab_size, max_sequence_length, embedding_dim):
    #跑的比較慢，準確率也滿高的
    #https://mlwhiz.com/blog/2019/03/09/deeplearning_architectures_text_classification/
    #https://www.kaggle.com/sanket30/cudnnlstm-lstm-99-accuracy
    #https://www.kaggle.com/suicaokhoailang/10-fold-lstm-with-attention-0-991-lb
    #無法用predict_classes
    inp = Input(shape=(max_sequence_length,))
    x = Embedding(num_classes, embedding_dim#, weights=[embedding_matrix]
                 ,trainable=False)(inp)
    '''
    Here 64 is the size(dim) of the hidden state vector as well as the output vector. Keeping return_sequence we want the output for the entire sequence. So what is the dimension of output for this layer?
        64*70(maxlen)*2(bidirection concat)
    CuDNNLSTM is fast implementation of LSTM layer in Keras which only runs on GPU
    '''
    
    x = Bidirectional(LSTM(256,dropout=0.4,recurrent_dropout=0.4,activation='relu', return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    att = Attention(max_sequence_length)(x)
    conc = concatenate([att,avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.5)(conc)
    outp = Dense(num_classes, activation="softmax")(conc)
    model = Model(inputs=inp, outputs=outp)
    print(model.summary())
    return model



In [55]:
def build_model(num_classes, vocab_size, max_sequence_length, embedding_dim):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, 
                        output_dim=256, 
                        input_length=max_sequence_length))
    model.add(Bidirectional(LSTM(128,implementation=2)))
#     model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

In [56]:
model = build_model(num_classes, NUM_WORDS,MAX_LEN,embedding_dim)

model_name = '1999-Fine-Tune'
lr = 2e-5
optimizer = adam(lr=lr,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 256)           1280000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 1,675,525
Trainable params: 1,675,525
Non-trainable params: 0
_________________________________________________________________


# 訓練模型

In [57]:
model_name = 'intent_model'
model_path = 'saved_models/{}.h5'.format(model_name)
epochs =10
batch_size = 20
train_ratio = 0.9


#model_path = './saved_models/DensetNet_0524001.h5'.format(model_name)

checkpoint = ModelCheckpoint(model_path, monitor='val_acc', save_best_only=True, verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.1, min_lr=0.000001, cooldown=1)

earlystop = EarlyStopping(monitor='val_loss', patience=6, verbose=1)



model_history = model.fit(x=x_train, y=y_one_hot,
                            batch_size=batch_size, 
                            epochs=epochs,
                            validation_split= 1 - train_ratio
                            #callbacks = [ checkpoint,earlystop]
                         )



Train on 5659 samples, validate on 629 samples
Epoch 1/10
5659/5659 [==============================] - 20s 3ms/step - loss: 0.8697 - acc: 0.6717 - val_loss: 0.6048 - val_acc: 0.7933
Epoch 2/10
5659/5659 [==============================] - 17s 3ms/step - loss: 0.3568 - acc: 0.8851 - val_loss: 0.5830 - val_acc: 0.8013
Epoch 3/10
3700/5659 [==================>...........] - ETA: 5s - loss: 0.1988 - acc: 0.9332

KeyboardInterrupt: 

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(model_history,'acc','val_acc')

In [ ]:
show_train_history(model_history,'loss','val_loss')

In [ ]:
# model.save(model_path)
x_test


In [ ]:
y_predict_name, y_predict, predict_arr,y_predict_probability = predict(model,x_test,mapping)

In [ ]:
#y_predict_probability2=y_predict_probability.astype(float)
#y_predict_probability3 = pd.Series(['{0:.2f}%'.format(val * 100) for val in y_predict_probability2])

In [ ]:
y_predict

In [ ]:
#df_test_y
#x_test
#x_test,train_word_index = covert_sequences(x_test,NUM_WORDS, MAX_LEN)


In [ ]:
def to_cat_name(x): 
        return mapping[x]
    
def predict(test):
#     model = load_model(model_path)
    y_predict_probability = model.predict(test)
    y_predict = model.predict_classes(test)

    return to_cat_name(y_predict), y_predict, y_predict_probability

y_predict_name, y_predict, y_predict_probability = predict(x_test)

In [ ]:
answer_name = to_cat_name(df_test_y)

In [ ]:
y_predict

In [ ]:
df_test_y

In [ ]:
predict_df = pd.DataFrame({'1_sentence':df_test_x.description_clean,'2_sentence':df_test_x.cut_words,
              '3_y_predict':y_predict,
              '4_y_predict_name':y_predict_name,
              '5_answer':df_test_y,
              '6_answer':mapping[df_test_y],
              '7_y_predict_probability':list(predict_arr),
              })
#print( format(y_predict_probability, '.3%'))
predict_df[:10]

In [ ]:
# accuracy
accuracy = 1 - (float(len(predict_df[predict_df['3_y_predict'] != predict_df['5_answer']])))/(len(predict_df))
print ('accuracy : {} !!!!!!!!'.format(accuracy))

In [ ]:
differ_ans=predict_df[predict_df['3_y_predict'] != predict_df['5_answer']]
differ_ans

In [ ]:
pd.DataFrame(differ_ans).to_csv('differ_ans_ab_Network2.csv')

In [ ]:
# confusion matrix
def to_cat_name(x): 
        return mapping[x]
pd.crosstab(np.array(y_predict_name), to_cat_name(np.array(df_test_y)),
            rownames=['3_y_predict'], colnames=['answer'])

In [ ]:
dfTest = pd.read_csv('Project_19999/data/19999_test_question_v1.csv')#usecols=['description','category_a', 'category_a_target']

In [ ]:
dfTest

In [ ]:
dfTest['cut_words'] = dfTest['description_clean'].apply(lambda s: cut_to_word(s.strip(),js))+','+dfTest['category_a_b'].apply(lambda s: cut_to_word(s.strip(),js))

In [ ]:
dfTest

In [ ]:
x_test_new,train_word_index_new = covert_sequences(dfTest.cut_words,NUM_WORDS, MAX_LEN)

In [ ]:
mapping

In [ ]:
y_test_new=dfTest.target

In [ ]:
from data_utils import *

In [ ]:
y_predict_name_new, y_predict_new, predict_arr_new,y_predict_probability_new = predict_class2(model,x_test_new,64,mapping)

In [ ]:
predict_df = pd.DataFrame({'1_sentence':dfTest.description_clean,'2_sentence':dfTest.cut_words,
              '3_y_predict':y_predict_new,
              '4_y_predict_name':y_predict_name_new,
              '5_answer':y_test_new,
              '6_answer':mapping[y_test_new],
              '7_y_predict_probability':list(predict_arr_new),
              })
#print( format(y_predict_probability, '.3%'))
predict_df[:14]

In [ ]:
pd.DataFrame(predict_df).to_csv('differ_test_new.csv')

In [ ]:
# def train(self,epochs=50):
#     print 'building model ...'
#     self.model = SentimentLSTM.build_model()

#     print 'loading data ...'
#     (text_train, rate_train), (text_test, rate_text) = self.load_data()

#     print 'training model ...'
#     self.model.fit(text_train, rate_train,batch_size=1000,epochs=epochs)
#     self.model.save('model/keras.model')
#     score = self.model.evaluate(text_test,rate_text)
#     print score

# def load_trained_model(self,path):
#     model = SentimentLSTM.build_model()
#     model.load_weights(path)
